<a href="https://colab.research.google.com/github/Lucy-Moctezuma/SFSU-CodeLab-Work-/blob/main/E.%20Coli%20Machine%20Learning%20Project/5_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Learning (Neural Network)**

## ***Objectives for this Notebook***
- Introducing the importance of feature scaling and basic concepts behind Neural Networks used in the context of tabular data.
- Create functions to implement Neural Networks into Moragadivand's dataset.

**Deep Learning or Neural Networks**  is a method within Machine Learning (ML) in which the computer learns to accomplish a task through trial and error by analyzing training samples. Neural networks are loosely inspired by how biological neurons are connected and signal each other. It's important to know that there are many kinds of neural networks that can be used for different kinds of data (images, sounds, etc) and there are specific names for different Neural network architectures (Convolutional Neural Networks, Recurrent Neural Networks, etc.). We will however focus only with basic Neural Network for tabular data (i.e. dataframes).

### **Parts of a Neural Network**

All neural networks despite of their function and kind of data they deal with have in essence these following parts.

![neuralnet.png](https://drive.google.com/uc?export=view&id=1df7Dq1LS9QTFOdLuvOWb1Et6IiMIq4FK)

**1) Input layer:** is the layer that we use to feed our initial data. These can be data tables, text, images, etc. In our case we would be working with a data table, and this layer will contain the same amount of nodes as there are feature columns in our dataset.

**2) Hidden layers:** are the ones that will further process the information they receive from the input layer. In the example above we have 2 hidden layers but the amount of layers can vary depending on the task.

**3) Output layer:** is the final layer where we get our predictions, because we are dealing with classification of just 2 classes we will have 2 nodes in this layer.

**4) Nodes:** are the the components of each layer and it represents a center where computation and mathematical equations determine what information is passed to the next layer. Nodes are connected to the following layers differently.

**5) Weights:** are values that are meant to show the strenght of the relationship between each node. The general idea is that a neural network starts with a random set of weights and then during training, the weights get updated in a trial and error fashion until it finds the best combination of weights that will yield the highest performing model.
**Notice that in the image above, every black arrow has its own weight**


### **1) Importing Packages needed**
For Neural Networks we will introduce some new python packages: **Keras** and **Tensorflow**. Both of these are widely used within the Python community to construct Neural Networks.

In [ ]:
# Data manipulation imports for ML
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

# Import packages for Neural Networks model
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import scale
from sklearn.utils.class_weight import compute_class_weight
import keras as keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

# Imports for model evaluation
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Imports for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils.vis_utils import plot_model

# Imports for file management
import os
from google.colab import drive
drive.mount('/content/drive')

**SEED SETTING**

There is a lot of randomnes that contribute to Neural Networks results. Nevertheless, a good Neural Network Model should not depend on the seed but the data used and the architechture, given this, metrics should not be extremely affected by seed setting. Therefore this randomness is actually desired and not a problem.

However, for the sake of this learning tutorial and for reproducibility of these results we will be fixing **seed_value: 42** for all the background computations. Below we list the different sources of randomnes:

- Within Environment
- Within Python language
- Within specific packages (numpy, tensorflow)
- When choosing splits for training and testing data
- Within Learning Algorythm:
  - Neural network sets random weights at the beggining of training
  - Some special layers such as Dropout layers introduce randomness

The code below will **seed_value: 42** globally for the environment, the **python** language and for 2 packages (**numpy and tensorflow**). We will also quickly indicate where we have set other seeds. Everytime a seed is set you will see `seed_value` within the code of this notebook.



In [ ]:
# Set a seed value
seed_value= 42

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

### **2) Loading CSV file and creating dataframes for each antibiotic**

As in all previous ML model Notebooks, We will load our original dataframe, which contains all antibiotic drug labels and all features (GYS) and then we will be creating a dataframe for each antibiotic using different functions.

#### **a) Loading CSV created from previous notebook**

**NOTE:** Code below is the same as in previous notebook

In [ ]:
# Loads csv file as a dataframe
filepath = '/content/drive/My Drive/EColi_ML_CSV_files/'

# reads csv file as a dataframe
All_Drugs_df = pd.read_csv(filepath+"EColi_Merged_dfs.csv", na_values="NaN")
All_Drugs_df.head()

#### **b) Changing "R" to 0 and "S" to 1  for Deep Learning Model**
Neural Networks work using numeric values, therefore we will be converting all our target labels into floats. The last line of code shows the recoded version of all the labels for the antibiotic drugs.

In [ ]:
# creating a list of antibiotic names
drug_list = All_Drugs_df.iloc[:,1:13].columns

# converts all S values into 1 for each antibiotic
for drug in drug_list:
  All_Drugs_df.loc[All_Drugs_df[drug] == "S", drug] = 1.0

# converts all R values into 0 for each antibiotic
for drug in drug_list:
  All_Drugs_df.loc[All_Drugs_df[drug] == "R", drug] = 0.0

# Checking at how S and R classes were recoded
All_Drugs_df.head()

### **3) Separating each Drug Dataframe into 4 sections : Training (features and labels) and Testing (features and labels)**

**NOTE:** Code below is the same as in previous notebook

**a) Creating Testing and Training datasets for each antibiotic drug**

 *Seed value was used in the train_test_split() function, to split the data consistentently, that is, the observations chosen to be part of the training chunk will be consistent regardless of how many times the code is ran.*



In [ ]:
# Separating each dataframe into Labels and Features for training and testing data.
# Our function uses the handy train_test_split() function.

def Split_train_test(drug):
  #here we make a list of the columns we want to keep: the column for the isolate, the column for the drug we are interested in and all features (starting from column 13).
  df_list = [All_Drugs_df[["Isolate",drug]], All_Drugs_df.iloc[:,13:]]

  #here we create a data frame with just the columns we wanted to keep.
  Drug_df = pd.concat(df_list, axis=1)

  #here we drop all rows with missing data
  Drug_df = Drug_df.dropna()

  # Creating a dictionary to store each antibiotic datasets
  Train_test_dic = {}

  # Defining the label columns
  labels = Drug_df[drug]

  # Defining features columns
  features = Drug_df.drop(columns=[drug])

  # Separating training (features and labels) and testing (features and labels) datasets
  features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=seed_value, stratify=labels)

  # storing each data chunk in a dictionary
  Train_test_dic['labels_train'] = labels_train
  Train_test_dic['features_train'] = features_train
  Train_test_dic['labels_test'] = labels_test
  Train_test_dic['features_test'] = features_test

  return Train_test_dic

In [ ]:
# Implementing the function Split_train_test() for AMC example
AMX_Train_test_dic = Split_train_test("AMX")

# checking the shape of each dataframe or series stored in the dictionary created for drug AMC
print("AMX")
for k, df in AMX_Train_test_dic.items():
  print(k, df.shape)
  # counting how many of the labels have susceptible versus resistant ones
  if k.startswith("labels"):
    print(df.value_counts())

Remember that our target values have been recoded. **0.0 is Resistance (R)** and **1.0 is Susceptible (S)**. And in our implementation we used the antibiotic AMX. The total number of training observations is 732.

### **4) Creating different combination of features before training**

Similar to previous notebooks, we have created a function that helps us choose independently what group of features we would like to train with (Gene Absence Presence or Absence (**G**), Year of Isolation (**Y**) and/or Population Structure (**S**). But for Neural Networks, we have added a subtle but important difference between this function and the prior ones created. In here we specifically ask our features to be **Scaled**.

**Scaling:** is a technique where we try to change the numeric values in all columns so that they share the same scale. Scaling is particularly useful for Neural Networks as it helps the model find the minima of the cost function quicker. There are different ways to rescale data, the two main ways are: [Normalization and Standarization](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/#Why_Should_We_Use_Feature_Scaling?).


In [ ]:
# making a list of combinations of data sources we would like to test in our ML models
combo_list = ['G', 'S', 'GY', 'SY', 'GS', 'GYS']

# making a function that creates different feature combinations of the predictor features
def combo_feat(features_df, drug, combo):

  # creating Year column filters for features_df
  year_filter = [col for col in features_df if col.startswith("Year")]
  year_feat = features_df[year_filter]

  # creating Population structure column filters for features_df
  pop_str_filter = [col for col in features_df if col.startswith("cutoff")]
  pop_struc_feat = features_df[pop_str_filter]

  # creating Gene precence column filters for features_df
  gene_presc_filter = [col for col in features_df.columns if col not in pop_str_filter and col not in year_filter and col != "Isolate"]
  gene_presc_feat = features_df[gene_presc_filter]

  if combo == 'G':
    df_list = [features_df['Isolate'], gene_presc_feat]
    G_feat_df = pd.concat(df_list, axis=1)
    G_feat_df = scale(G_feat_df.drop(columns=['Isolate']))
    return G_feat_df

  if combo == 'S':
    df_list = [features_df['Isolate'], pop_struc_feat]
    S_feat_df = pd.concat(df_list, axis=1)
    S_feat_df = scale(S_feat_df.drop(columns=['Isolate']))
    return S_feat_df

  if combo == 'GY':
    df_list = [features_df['Isolate'], gene_presc_feat, year_feat]
    GY_feat_df = pd.concat(df_list, axis=1)
    GY_feat_df = scale(GY_feat_df.drop(columns=['Isolate']))
    return GY_feat_df

  if combo == 'SY':
    df_list = [features_df['Isolate'], pop_struc_feat, year_feat]
    SY_feat_df = pd.concat(df_list, axis=1)
    SY_feat_df = scale(SY_feat_df.drop(columns=['Isolate']))
    return SY_feat_df

  if combo == 'GS':
    df_list = [features_df['Isolate'], gene_presc_feat, pop_struc_feat]
    GS_feat_df = pd.concat(df_list, axis=1)
    GS_feat_df = scale(GS_feat_df.drop(columns=['Isolate']))
    return GS_feat_df

  if combo == 'GYS':
    df_list = [features_df['Isolate'], gene_presc_feat, year_feat, pop_struc_feat]
    GYS_feat_df = pd.concat(df_list, axis=1)
    GYS_feat_df = scale(GYS_feat_df.drop(columns=['Isolate']))
    return GYS_feat_df

In the function we created, we scale the features using **scale()**, which actually performs an **Standarization** (where values are mean centered and distance from mean is measured in standard deviations). Without scaling values, the Neural Network could get biased towards features that have a higher order of magnitude.

Below we can see an example of the difference between raw values and standarized values for two columns. Notice that in raw residuals the standard deviations are not the same. Without scaling its hard to compare which variable of the two might be more important as their ranges differ.



In [ ]:
# features before scaling: Raw values
AMX_Train_test_dic['features_train'][['cutoff_25459','cutoff_2']].describe()

Below we have the scaled features and we can notice the following:
- All feature descriptives get smaller
- The means and ranges are still different like in our unscaled data
- The standard deviations are the same now.

Now we can compare the values based on the number of standard deviations away from their mean. We are able to tell within the same observation and for both variables whether they represent a typical value (sd closer to mean) or an extreme value (sd further from mean). Now our model has an easier way to tell which of the two features might be more relevant.

It's important to know that feature scaling may not always make a difference, it really depends on the type of Machine Learning algorithm you choose. For example, we did not do it for previous notebooks or tree based methods, but we did it in here. You can read more about it in this summary [article](https://towardsdatascience.com/what-is-feature-scaling-why-is-it-important-in-machine-learning-2854ae877048#:~:text=Feature%20scaling%20is%20the%20process,need%20to%20perform%20feature%20scaling.)

In [ ]:
# features after scaling: standarization
feat_scaled = scale(AMX_Train_test_dic['features_train'][['cutoff_25459','cutoff_2']])
feat_scaled = pd.DataFrame(feat_scaled, columns=[['cutoff_25459_(scaled)','cutoff_2_(scaled)']])
feat_scaled.describe()

Now we are ready to implement our function in a complete training dataset. Instead of having just 2 columns being scaled, it will do it for all the feature columns in the G and S datasets for the antibiotic "AMX".

In [ ]:
# Implementing combo_feat() function created for training data
AMX_GS_train_array = combo_feat(AMX_Train_test_dic['features_train'],"AMX","GS")

# Each list within the array represents a row
AMX_GS_train_array

### **5) Creating Deep Learning model and training it per feature combination**
Neural Networks contain many hyperparameters we can tune, the input layer in this case doesn't need to be set up as it is determined by the amount of features we are using to train. But we will be setting the hidden layers and the final output layer.:
- The number of nodes for the hidden first layer.
- The number of hidden layers we want after the first one.
- The number of nodes for the subsequent layers
- The dropout rate for hidden layers
- The learning rate


In [ ]:
# Number of nodes on first hidden layer
firstlayer = 200

# Number of hidden layers
numblayer = 2

# Number of nodes on each hidden layer
interlayer = 100

# Dropout rate
dropout = 0.8  # default was 0.8

# Learning rate
lr= 0.001

After setting some of our Neural Network Paramenters, we can create our function. The other parameters and the architecture of the model is based on how Moragadivand's had it set up for the paper (number of layers and nodes, other parameter settings, etc).

The general structure of how each layer is created is shown below in the pseudo code, we will also briefly define some vocabulary used in our specific code:

<font color="grey">`Model.add(Layertype(#_output_weights, activation= 'specific function', input_shape=#_input_weights)`

For Neural Network *Architechture*:
- **Dense:** This is a kind of layer, where each node from a dense layer receives conexions from all the previous nodes, for this reason they are often called **"fully connected"**.

- **Dropout:** This is a special type of layer with the sole function to block a percentage of weights that pass from one layer to the next. A Dropout rate assigns a percentage of randomly selected neurons in a layer to become inactive, meaning their contribution in the forward pass is not taken into account and therefore the weights in the backward pass towards these dropout neurons do not get updated. This is considered to be a **Regularization method**, meaning it is done so that our model doesn't follow the training data too closely and is able to generalize better for unseen data.

- **Activation function:** is a parameter we can pass for some types of layers, it is essentially a function that computes the output for a layer. For example, for Dense layers we have ["Relu"](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)), whereas for dropout layers we dont have one, as they don't need to compute an output, they just block outputs from the layer before from going to the next layer, by converting some of these to 0.
***
For Neural Network *Training behavior*:

<font color="grey">`EarlyStopping(patience=# training epochs)`

- **Early Stopping:** is a parameter that essentially stops the training based on the maximum number of epochs accepted before stopping the training if no improvement is seen.

<font color="grey">`Model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lr) ,loss= 'binary_crossentropy', metrics=['accuracy'])`

- **Learning rate**: is a value between 0 and 1, that determines basically how fast our model will try to learn. The higher the learning rate, the less number of epochs (training cycles) is required. This is the most important parameter to tune in a Neural Network. Too large and it will converge too quickly and provide a suboptimal result, too small and it will get stuck.

- **Loss function**: there are many loss functions we can use depending on the type of data and task. A loss function is essentially a method used to evaluate how well our algorithm models our training data. Our objective is to minimize it since its essentially our errors. In our case we use:  *binary_crossentropy* (a mathematical function that measures the difference between predicted probabilities and actual labels in classification task with only 2 classes)

- **Metrics**: this is just a measurement we can look at in order to judge the performance of the model while training, and as our loss function is being optimized.
***
*Seed_values were set here for the initial weights prior to training and for every Dropout layer in the Neural Network*

In [ ]:
# creating Deep Learning model function
def run_DL(feat_train_df, lab_train, drug, combo, view_training = True):
  print(drug +" Training combo: "+ combo)

  # Reweighting classes due to imbalanced dataset
  class_labels = np.unique(lab_train)
  reweight = compute_class_weight(class_weight='balanced', classes=class_labels, y=lab_train)

  # Constructing Deep Learning model

  # Choose the type of Neural Network you want create
  DL = Sequential()

  # Adding the first hidden layer.
  DL.add(Dense(int(firstlayer),activation='relu', input_shape=(feat_train_df.shape[1],)))

  # Adding the dropout layer for the first layer.
  DL.add(Dropout(dropout, input_shape=(feat_train_df.shape[1],), seed=seed_value))

  # Create more hidden layers
  for i in range(1,int(numblayer)):
      DL.add(Dense(int(interlayer),activation='relu'))

      # For each of the hidden layers also create dropout
      DL.add(Dropout(dropout, seed=seed_value))

  # Create the output layer that consist only on 2 nodes (we have 2 classes)
  DL.add(Dense(2, activation = 'softmax', kernel_initializer=keras.initializers.glorot_uniform(seed=seed_value)))

  # Additional parameters for training (Early stopping)
  early_stopping_monitor= EarlyStopping(patience=50)

  # Compiling model created
  DL.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lr) ,loss= 'binary_crossentropy', metrics=['accuracy'])

  # Training with the neural network created
  DL_history = DL.fit(feat_train_df, to_categorical(lab_train), validation_split=0.2, callbacks= [early_stopping_monitor],epochs=10, batch_size=128, class_weight=dict(enumerate(reweight)))

  # visualizing each training
  if view_training == True:

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle('Training History')
    print(DL_history.history.keys())
    ax1.plot(DL_history.history['accuracy'])
    ax1.plot(DL_history.history['val_accuracy'])
    ax1.set_title('model accuracy')
    ax1.set(ylabel='accuracy', xlabel='epochs')

    ax2.plot(DL_history.history['loss'])
    ax2.plot(DL_history.history['val_loss'])
    ax2.set_title('model loss')
    ax2.set(ylabel='loss', xlabel='epochs')
    ax2.legend(['train', 'validation'], loc='best')
    fig.tight_layout()
    filepath='/content/drive/My Drive/EColi_ML_Plots/'
    plt.savefig(filepath + 'DL_' + drug + '_' + combo + '_' + 'training_history.jpg',dpi=400, bbox_inches="tight")

  return DL



<font color="grey">`Model.fit(features, to_categorical(labels), validation_split = percent of training data used for validation, callbacks= [early_stopping_monitor],epochs=# of training cycles, batch_size= # observations fed at a time))`

A Neural Network, is trained in 2 general stages:

**1) Feedforward**: First the observations start on the input layer go through all the hidden layers and then it produces an initial prediction in the output layer.

**2) Backpropagation**: An error is calculated between our actual training labels and the output we got from the Feedforward process. It then adjusts the weights from output layer back to input layer in order to minimize the errors.
***
**Epoch:** Is one cycle of training where the entire dataset goes through a Feedforward process and a Backpropagation one. We have set a total of 10 epochs.

**Batch:** Because feeding the entire dataset might be computationally expensive, it is a common practice to feed our neural network data in batches, we have chosen a batch size of 128.

**Validation Data:** Is a chunk of the training data that is reserved to check if our model is generalizing well to all the training data. This is different from the testing data, because it is being used during model training.

Our total number of observations is 732, and our batch size is 128. Therefore, in one epoch we will have 732/128, so about 5 batches per epoch. Below we will implement our function for the antibiotic AMX and using only the features (G and S).

In [ ]:
# implementing run_DL() for specific drug feature combination dataframe
DL_AMX_GS_model = run_DL(AMX_GS_train_array, AMX_Train_test_dic['labels_train'],"AMX","GS")

- In the graph above we can see that training and validation data have an upwards direction, meaning that we can see that accuracies for both training and validation are going up, meaning there is learning happening.
- We notice that the **training set** has a lower overall accuracy and a higher loss than the **validation set**, this is not usually the case, we would normally observed the reversed pattern. Because the validation is supposed to simulate unseen data it should have lower accuracies and higher loss, but because **Dropout layer** is implemented, regularization happens and this generalizability makes the validation metrics better. You can test this by lowering the dropout rates in a new session of this notebook, in doing so the lines will beggin to overlap and we will start to see more of a reversed pattern. The current dropout rate is at 0.8.  

In the code below we can graph our Neural Network model architechture:


In [ ]:
# plotting the schematics of our model
plot_model(DL_AMX_GS_model, to_file='/content/drive/MyDrive/EColi_ML_Plots/DL_AMX_GS_model_Architechture.jpg', show_shapes=True, show_layer_activations=True, rankdir='LR')

We can see in the figure that the Input layers input contains 18269 nodes, this is because each node pertains to one feature. Remember that this model was trained with only G and S dataset features. Our input layer nodes will change depending on out training data.

For each layer we can see the following information displayed:
- The name of the layer (example: dense_input)
- The type of neurons the layer contains (example: Dense, Droput)
- The activation function the layer uses (example: softmax, relu)
- The amount of connections for the nodes it receives (input) - The amount of connections leaving the node (output). For dropout layers, we see that input and output shape are the same. However, when a weight gets filtered here, the amount passed is 0. Also, which neurons' weights become 0 changes every cycle of training, which is why everytime we run the code it will give you slightly different results.


### **6) Making predictions from Deep Learning model**

After training our Neural network we are now ready to make predictions.

In [ ]:
# creating a function using the model created and trained and the feature combinations from testing data
def predict(DL_combo_Model, features_test):
  labels_pred = DL_combo_Model.predict(features_test)
  labels_pred = np.argmax(labels_pred, axis=1)
  return labels_pred

We will now use our **combo_feat()** function to choose the appropiate data feature combination for Testing. In the example below, we use the combination of features G and S.

In [ ]:
# Implementing combo_feat() function created for testing data
AMX_GS_test_array = combo_feat(AMX_Train_test_dic['features_test'],"AMX","GS")

# Each list within the array represents a row
print(AMX_GS_test_array)
print("Number of rows: ",len(AMX_GS_test_array))

Then we will implement our **predict()** function and checkout our predictions!

Remember that 0 means **Resistant (R)** and 1 means **Susceptible (S)**

In [ ]:
# Implementation of the predict() function using the feature combination "GS"
AMX_GS_labels_pred = predict(DL_AMX_GS_model,AMX_GS_test_array)

# observe how many predictions were made for each category "R"=0 and "S"=1
print("Labels predicted: ")
print(AMX_GS_labels_pred)

### **7) Evaluating our model using a confusion matrix and metrics**

Similarly to the previous notebook, we will evaluate our Deep learning model by using a Confusion Matrix and respective metrics. Below is a quick review of these, remember that there is one Accuracy score, but Recall and Presicion should have as many sets as classes our model its trained to predict:

|<font size=3>Metrics|<font size=3>General formula| <font size=3>Formula for 2 classes|
|--|:-:|:-:
|<font size=3>**Accuracy**|<font size=3>$\frac{Correctly \ classified}{All \ Predicted}$|<font size=3>$\frac{TP + TN}{TP + TN + FN + FP}$|
|<font size=3>**R Recall:**|<font size=3>$\frac{Correctly \ classified \ as \ R}{All \ Actual \ R}$|<font size=3>$\frac{TP}{TP + FN}$|
|<font size=3>**R Precision:**|<font size=3>$\frac{Correctly \ classified \ as \ R}{All \ Predicted \ R}$|<font size=3>$\frac{TP}{TP + FP}$|

In [ ]:
# Creating a function that evaluates our model using our actual and predicted data
def evaluate(DL_combo_model, features_test, labels_test, labels_pred, cf= True, show_results= True):

  labels_test = np.asfarray(labels_test,float)
  score = DL_combo_model.evaluate(features_test, to_categorical(labels_test)) # only take accuracy

  labels = unique_labels(labels_test, labels_pred)
  inp = precision_recall_fscore_support(labels_test, labels_pred, labels=labels, average=None)
  report = np.asarray(inp).ravel().tolist()
  report= pd.DataFrame(report, index = ['PRC_R','PRC_S','RCL_R','RCL_S','FSc_R','FSc_S','Sc_R','Sc_S'])
  report = report.transpose()

  if cf == True:
    cm = confusion_matrix(labels_test, labels_pred, labels=labels, sample_weight=None)
    labels= np.where(labels<1,"R","S")
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot()
    plt.show()
  if show_results == True:
    print("Results")
    print('Accuracy:',score[1])
    print('R recall:',report['RCL_R'][0])
    print('S recall:',report['RCL_S'][0])
    print('R precision:',report['PRC_R'][0])
    print('S precision:',report['PRC_S'][0])
  return [score[1], report['RCL_R'][0], report['RCL_S'][0], report['PRC_R'][0], report['PRC_S'][0]]

In [ ]:
# implementing the evaluate() function
Model_Report = evaluate(DL_AMX_GS_model,AMX_GS_test_array, AMX_Train_test_dic['labels_test'],AMX_GS_labels_pred)

### **8) Use all functions and evaluate every drug in every feature combination!**
**NOTE:** Code below is the same as in previous notebook

#### **a) Lets recall the list of drugs we have available and the combination of features we are interested in**

In [ ]:
# let's check all drugs
print(drug_list)

# let's see all combinations we are interested in
print(combo_list)

#### **b) Create a loop that will go through all our functions using the lists above**

In [ ]:
# Lets use all our functions this time and save our report into a single data structure
DL_model_metrics = {}

for drug in drug_list:
  print(drug)
  Test_Train_dic = Split_train_test(drug) # splits each drug df into a dictionary with testing and training data
  for combo in combo_list:
    # Training each drug_combo features
    labels_train = Test_Train_dic["labels_train"]
    features_train = combo_feat(Test_Train_dic["features_train"], drug, combo) # create corresponding feature_df for training
    DL_combo_model = run_DL(features_train, labels_train, drug, combo, view_training = False) # runs deep learning model using the corresponding training feature_df

    # Predicting each drug_combo features
    features_test = combo_feat(Test_Train_dic["features_test"], drug, combo) # create corresponding feature_df for testing
    labels_pred = predict(DL_combo_model, features_test) # generate predictions based on the feature combination tested

    # Evaluating our models
    labels_test = Test_Train_dic["labels_test"]
    report = evaluate(DL_combo_model,features_test, labels_test, labels_pred, cf=False, show_results= False)
    DL_model_metrics[drug+"_"+combo] = report

    print(report)

#### **b) Store the metrics report for all drugs and features combinations as a csv file**

In [ ]:
# convert dictionary into a dataframe
DL_metrics = pd.DataFrame.from_dict(DL_model_metrics, orient='index',columns=["Accuracy", "R_recall", "S_recall", "R_precision", "S_precision"]).reset_index()
DL_metrics = DL_metrics.rename(columns = {'index':'Drug_combo'})

# saving our metric results into a CSV file
DL_metrics.to_csv(filepath+"DL_metrics_df.csv", index= False)
DL_metrics


Congratulations for making it this far! We have finally finished creating all our Machine Learning Models and saved the results in different dataframes. Our [Last Notebook](https://colab.research.google.com/drive/1IH7yKbPY0jVUOplj6hdiRM3k470iaabG?usp=sharing) will now create a graph that will help us pick the best results for each of our antibiotics.